In [3]:
from confluent_kafka import Producer, Consumer
import json

In [2]:
def read_config():
  # reads the client configuration from client.properties
  # and returns it as a key-value map
  config = {}
  with open("client.properties") as fh:
    for line in fh:
      line = line.strip()
      if len(line) != 0 and line[0] != "#":
        parameter, value = line.strip().split('=', 1)
        config[parameter] = value.strip()
  return config

In [15]:
def delivery_report(err, msg):
    # Callback function to handle delivery report
    if err is not None:
        print(f"Message delivery failed: {err}")
    else:
        print(
            f"Message delivered to {msg.topic()} [{msg.partition()}] at offset {msg.offset()}"
        )

In [17]:
def produce(topic, config):
    # creates a new producer instance
    producer = Producer(config)

    with open("order_details.json", "r") as file:
        order_data = json.load(file)

    try:
        # ensure the producer is connected to the broker
        producer.poll(0)

        for order in order_data:
            value = order
            key = value["OrderID"]

            # produce a message to the specified topic
            producer.produce(
                topic,
                key=str(key).encode("utf-8"),
                value=json.dumps(value).encode("utf-8"),
                callback=delivery_report,
            )

    except Exception as e:
        print(f"Error producing message: {e}")

    finally:
        # send any outstanding or buffered messages to the Kafka broker
        producer.flush()

In [19]:
def consume(topic, config):
  # sets the consumer group ID and offset
  config["group.id"] = "order-group-1"
  config["auto.offset.reset"] = "earliest"

  # creates a new consumer instance
  consumer = Consumer(config)

  # subscribes to the specified topic
  consumer.subscribe([topic])
  
  try:
    while True:
      # consumer polls the topic and prints any incoming messages
      msg = consumer.poll(1.0)
      if msg is not None and msg.error() is None:
        key = msg.key().decode("utf-8")
        value = msg.value().decode("utf-8")
        print(f"Consumed message from topic {topic}: key = {key} value = {value}")
  except KeyboardInterrupt:
    pass
  finally:
    # closes the consumer connection
    consumer.close()

In [20]:
def main():
  config = read_config()
  topic = "ecommerce"

  produce(topic, config)
  consume(topic, config)


main()

Message delivered to ecommerce [0] at offset 10
Message delivered to ecommerce [0] at offset 11
Message delivered to ecommerce [0] at offset 12
Message delivered to ecommerce [0] at offset 13
Message delivered to ecommerce [0] at offset 14
Message delivered to ecommerce [0] at offset 15
Message delivered to ecommerce [0] at offset 16
Message delivered to ecommerce [0] at offset 17
Message delivered to ecommerce [0] at offset 18
Message delivered to ecommerce [0] at offset 19
Message delivered to ecommerce [2] at offset 8
Message delivered to ecommerce [2] at offset 9
Message delivered to ecommerce [2] at offset 10
Message delivered to ecommerce [2] at offset 11
Message delivered to ecommerce [2] at offset 12
Message delivered to ecommerce [2] at offset 13
Message delivered to ecommerce [2] at offset 14
Message delivered to ecommerce [1] at offset 7
Message delivered to ecommerce [1] at offset 8
Message delivered to ecommerce [1] at offset 9
Message delivered to ecommerce [1] at offset 